# lab_results.ipynb
Author: UFO Software, LLC<br>
Created: Sunday, February 10, 2021 13:09<br>

License GPLv3+: GNU GPL version 3 or later <https://gnu.org/licenses/gpl.html> <br>
This is free software: you are free to change and redistribute it.<br>
There is NO WARRANTY, to the extent permitted by law.<br>
<br>
Reads in the LabResults_0.csv file from the [December 2020 WA State Traceability Data](https://lcb.app.box.com/s/fnku9nr22dhx04f6o646xv6ad6fswfy9?page=1) and drops the columns that have been depreciated.


In [8]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

# change the file path to match where your data is stored
file_path = Path('dec-2020')
pd.set_option('display.max_columns', None)

In [9]:
# parse date columns
date_cols = ['created_at',
             'deleted_at',
             'updated_at',
             'tested_at',
             'received_at']
lab_results_df = pd.read_csv(file_path / 'LabResults_0.csv', sep = '\t', encoding = 'utf-16', parse_dates = date_cols)
lab_results_df.head()

/Users/alienfoo/anaconda3/envs/cds_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3156: DtypeWarning: Columns (4,12,13,14,47,70,79,82,83,88,89,113,119,182) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,global_id,created_at,mme_id,user_id,external_id,deleted_at,inventory_id,updated_at,status,testing_status,batch_id,for_mme_id,parent_lab_result_id,og_parent_lab_result_id,copied_from_lab_id,lab_user_id,tested_at,received_at,type,foreign_matter,moisture_content_percent,herbicides_ppm,growth_regulators_ppm,cannabinoid_status,cannabinoid_editor,cannabinoid_d9_thca_percent,cannabinoid_d9_thca_mg_g,cannabinoid_d9_thc_percent,cannabinoid_d9_thc_mg_g,cannabinoid_d8_thc_percent,cannabinoid_d8_thc_mg_g,cannabinoid_thcv_percent,cannabinoid_thcv_mg_g,cannabinoid_cbd_percent,cannabinoid_cbd_mg_g,cannabinoid_cbda_percent,cannabinoid_cbda_mg_g,cannabinoid_cbdv_percent,cannabinoid_cbdv_mg_g,cannabinoid_cbg_percent,cannabinoid_cbg_mg_g,cannabinoid_cbga_percent,cannabinoid_cbga_mg_g,cannabinoid_cbc_percent,cannabinoid_cbc_mg_g,cannabinoid_cbn_percent,cannabinoid_cbn_mg_g,terpenoid_status,terpenoid_editor,terpenoid_bisabolol_percent,terpenoid_bisabolol_mg_g,terpenoid_humulene_percent,terpenoid_humulene_mg_g,terpenoid_pinene_percent,terpenoid_pinene_mg_g,terpenoid_terpinolene_percent,terpenoid_terpinolene_mg_g,terpenoid_b_caryophyllene_percent,terpenoid_b_caryophyllene_mg_g,terpenoid_b_myrcene_percent,terpenoid_b_myrcene_mg_g,terpenoid_b_pinene_percent,terpenoid_b_pinene_mg_g,terpenoid_caryophyllene_oxide_percent,terpenoid_caryophyllene_oxide_mg_g,terpenoid_limonene_percent,terpenoid_limonene_mg_g,terpenoid_linalool_percent,terpenoid_linalool_mg_g,microbial_status,microbial_editor,microbial_total_viable_plate_count_cfu_g,microbial_total_yeast_mold_cfu_g,microbial_total_coliform_cfu_g,microbial_bile_tolerant_cfu_g,microbial_pathogenic_e_coli_cfu_g,microbial_salmonella_cfu_g,microbial_aerobic_bacteria_cfu_g,mycotoxin_status,mycotoxin_editor,mycotoxin_aflatoxins_ppb,mycotoxin_ochratoxin_ppb,metal_status,metal_editor,metal_arsenic_ppm,metal_cadmium_ppm,metal_lead_ppm,metal_mercury_ppm,pesticide_status,pesticide_editor,pesticide_abamectin_ppm,pesticide_acequinocyl_ppm,pesticide_bifenazate_ppm,pesticide_bifentrin_ppm,pesticide_captan_ppm,pesticide_cyfluthrin_ppm,pesticide_cypermethrin_ppm,pesticide_dimethomorph_ppm,pesticide_etoxazole_ppm,pesticide_fenhexamid_ppm,pesticide_flonicamid_ppm,pesticide_fludioxonil_ppm,pesticide_imidacloprid_ppm,pesticide_myclobutanil_ppm,pesticide_pcnb_ppm,pesticide_piperonyl_butoxide_ppm,pesticide_pyrethrin_ppm,pesticide_spinetoram_ppm,pesticide_spinosad_ppm,pesticide_spirotetramet_ppm,pesticide_thiamethoxam_ppm,pesticide_trifloxystrobin_ppm,solvent_status,solvent_editor,solvent_butanes_ppm,solvent_heptane_ppm,solvent_propane_ppm,notes,thc_percent,intermediate_type,moisture_content_water_activity_rate,solvent_acetone_ppm,solvent_benzene_ppm,solvent_cyclohexane_ppm,solvent_chloroform_ppm,solvent_dichloromethane_ppm,solvent_ethyl_acetate_ppm,solvent_hexanes_ppm,solvent_isopropanol_ppm,solvent_methanol_ppm,solvent_pentanes_ppm,solvent_toluene_ppm,solvent_xylene_ppm,solvent_heptanes_ppm,pesticide_acephate_ppm,pesticide_acetamiprid_ppm,pesticide_aldicarb_ppm,pesticide_azoxystrobin_ppm,pesticide_bifenthrin_ppm,pesticide_boscalid_ppm,pesticide_carbaryl_ppm,pesticide_carbofuran_ppm,pesticide_chlorantraniliprole_ppm,pesticide_chlorfenapyr_ppm,pesticide_chlorpyrifos_ppm,pesticide_clofentezine_ppm,pesticide_daminozide_ppm,pesticide_ddvp_dichlorvos_ppm,pesticide_diazinon_ppm,pesticide_dimethoate_ppm,pesticide_ethoprophos_ppm,pesticide_etofenprox_ppm,pesticide_fenoxycarb_ppm,pesticide_fenpyroximate_ppm,pesticide_fipronil_ppm,pesticide_hexythiazox_ppm,pesticide_imazalil_ppm,pesticide_kresoxim_methyl_ppm,pesticide_malathion_ppm,pesticide_metalaxyl_ppm,pesticide_methiocarb_ppm,pesticide_methomyl_ppm,pesticide_methyl_parathion_ppm,pesticide_mgk_264_ppm,pesticide_naled_ppm,pesticide_oxamyl_ppm,pesticide_paclobutrazol_ppm,pesticide_permethrinsa_ppm,pesticide_phosmet_ppm,pesticide_prallethrin_ppm,pesticide_propiconazole_ppm,pesticide_propoxur_ppm,pesticide_pyrethrinsbc_ppm,pesticide_pyridaben_ppm,pesticide_spiromesifen_ppm,pesticide_spirotet

In [10]:
lab_results_df.tail()

,global_id,created_at,mme_id,user_id,external_id,deleted_at,inventory_id,updated_at,status,testing_status,batch_id,for_mme_id,parent_lab_result_id,og_parent_lab_result_id,copied_from_lab_id,lab_user_id,tested_at,received_at,type,foreign_matter,moisture_content_percent,herbicides_ppm,growth_regulators_ppm,cannabinoid_status,cannabinoid_editor,cannabinoid_d9_thca_percent,cannabinoid_d9_thca_mg_g,cannabinoid_d9_thc_percent,cannabinoid_d9_thc_mg_g,cannabinoid_d8_thc_percent,cannabinoid_d8_thc_mg_g,cannabinoid_thcv_percent,cannabinoid_thcv_mg_g,cannabinoid_cbd_percent,cannabinoid_cbd_mg_g,cannabinoid_cbda_percent,cannabinoid_cbda_mg_g,cannabinoid_cbdv_percent,cannabinoid_cbdv_mg_g,cannabinoid_cbg_percent,cannabinoid_cbg_mg_g,cannabinoid_cbga_percent,cannabinoid_cbga_mg_g,cannabinoid_cbc_percent,cannabinoid_cbc_mg_g,cannabinoid_cbn_percent,cannabinoid_cbn_mg_g,terpenoid_status,terpenoid_editor,terpenoid_bisabolol_percent,terpenoid_bisabolol_mg_g,terpenoid_humulene_percent,terpenoid_humulene_mg_g,terpenoid_pinene_percent,terpenoid_pinene_mg_g,terpenoid_terpinolene_percent,terpenoid_terpinolene_mg_g,terpenoid_b_caryophyllene_percent,terpenoid_b_caryophyllene_mg_g,terpenoid_b_myrcene_percent,terpenoid_b_myrcene_mg_g,terpenoid_b_pinene_percent,terpenoid_b_pinene_mg_g,terpenoid_caryophyllene_oxide_percent,terpenoid_caryophyllene_oxide_mg_g,terpenoid_limonene_percent,terpenoid_limonene_mg_g,terpenoid_linalool_percent,terpenoid_linalool_mg_g,microbial_status,microbial_editor,microbial_total_viable_plate_count_cfu_g,microbial_total_yeast_mold_cfu_g,microbial_total_coliform_cfu_g,microbial_bile_tolerant_cfu_g,microbial_pathogenic_e_coli_cfu_g,microbial_salmonella_cfu_g,microbial_aerobic_bacteria_cfu_g,mycotoxin_status,mycotoxin_editor,mycotoxin_aflatoxins_ppb,mycotoxin_ochratoxin_ppb,metal_status,metal_editor,metal_arsenic_ppm,metal_cadmium_ppm,metal_lead_ppm,metal_mercury_ppm,pesticide_status,pesticide_editor,pesticide_abamectin_ppm,pesticide_acequinocyl_ppm,pesticide_bifenazate_ppm,pesticide_bifentrin_ppm,pesticide_captan_ppm,pesticide_cyfluthrin_ppm,pesticide_cypermethrin_ppm,pesticide_dimethomorph_ppm,pesticide_etoxazole_ppm,pesticide_fenhexamid_ppm,pesticide_flonicamid_ppm,pesticide_fludioxonil_ppm,pesticide_imidacloprid_ppm,pesticide_myclobutanil_ppm,pesticide_pcnb_ppm,pesticide_piperonyl_butoxide_ppm,pesticide_pyrethrin_ppm,pesticide_spinetoram_ppm,pesticide_spinosad_ppm,pesticide_spirotetramet_ppm,pesticide_thiamethoxam_ppm,pesticide_trifloxystrobin_ppm,solvent_status,solvent_editor,solvent_butanes_ppm,solvent_heptane_ppm,solvent_propane_ppm,notes,thc_percent,intermediate_type,moisture_content_water_activity_rate,solvent_acetone_ppm,solvent_benzene_ppm,solvent_cyclohexane_ppm,solvent_chloroform_ppm,solvent_dichloromethane_ppm,solvent_ethyl_acetate_ppm,solvent_hexanes_ppm,solvent_isopropanol_ppm,solvent_methanol_ppm,solvent_pentanes_ppm,solvent_toluene_ppm,solvent_xylene_ppm,solvent_heptanes_ppm,pesticide_acephate_ppm,pesticide_acetamiprid_ppm,pesticide_aldicarb_ppm,pesticide_azoxystrobin_ppm,pesticide_bifenthrin_ppm,pesticide_boscalid_ppm,pesticide_carbaryl_ppm,pesticide_carbofuran_ppm,pesticide_chlorantraniliprole_ppm,pesticide_chlorfenapyr_ppm,pesticide_chlorpyrifos_ppm,pesticide_clofentezine_ppm,pesticide_daminozide_ppm,pesticide_ddvp_dichlorvos_ppm,pesticide_diazinon_ppm,pesticide_dimethoate_ppm,pesticide_ethoprophos_ppm,pesticide_etofenprox_ppm,pesticide_fenoxycarb_ppm,pesticide_fenpyroximate_ppm,pesticide_fipronil_ppm,pesticide_hexythiazox_ppm,pesticide_imazalil_ppm,pesticide_kresoxim_methyl_ppm,pesticide_malathion_ppm,pesticide_metalaxyl_ppm,pesticide_methiocarb_ppm,pesticide_methomyl_ppm,pesticide_methyl_parathion_ppm,pesticide_mgk_264_ppm,pesticide_naled_ppm,pesticide_oxamyl_ppm,pesticide_paclobutrazol_ppm,pesticide_permethrinsa_ppm,pesticide_phosmet_ppm,pesticide_prallethrin_ppm,pesticide_propiconazole_ppm,pesticide_propoxur_ppm,pesticide_pyrethrinsbc_ppm,pesticide_pyridaben_ppm,pesticide_spiromesifen_ppm,pesticide_spirotet

In [11]:
#drop decpreciated columns
decpreciated_cols = ['cannabinoid_cbc_mg_g',
                     'cannabinoid_cbc_percent',
                     'cannabinoid_cbdv_mg_g',
                     'cannabinoid_cbga_mg_g',
                     'cannabinoid_cbga_percent',
                     'cannabinoid_cbn_mg_g',
                     'cannabinoid_cbn_percent',
                     'cannabinoid_thcv_mg_g',
                     'cannabinoid_thcv_percent',
                     'global_for_inventory_id',
                     'for_mme_id',
                     'herbicides_ppm',
                     'high_cbd_flag',
                     'high_thc_flag',
                     'microbial_aerobic_bacteria_cfu_g',
                     'microbial_total_coliform_cfu_g',
                     'microbial_total_viable_plate_count_cfu_g',
                     'microbial_total_yeast_mold_cfu_g',
                     'pesticide_bifentrin_ppm',
                     'pesticide_captan_ppm',
                     'pesticide_dimethomorph_ppm',
                     'pesticide_fenhexamid_ppm',
                     'pesticide_pcnb_ppm',
                     'pesticide_piperonyl_butoxide_ppm',
                     'pesticide_pyrethrin_ppm',
                     'pesticide_spinetoram_ppm',
                     'received_at',
                     'solvent_heptanes_ppm',
                     'terpenoid_b_caryophyllene_mg_g',
                     'terpenoid_b_caryophyllene_percent',
                     'terpenoid_b_myrcene_mg_g',
                     'terpenoid_b_myrcene_percent',
                     'terpenoid_b_pinene_mg_g',
                     'terpenoid_b_pinene_percent',
                     'terpenoid_bisabolol_mg_g',
                     'terpenoid_bisabolol_percent',
                     'terpenoid_caryophyllene_oxide_mg_g',
                     'terpenoid_caryophyllene_oxide_percent',
                     'terpenoid_editor',
                     'terpenoid_humulene_mg_g',
                     'terpenoid_humulene_percent',
                     'terpenoid_limonene_mg_g',
                     'terpenoid_limonene_percent',
                     'terpenoid_linalool_mg_g',
                     'terpenoid_linalool_percent',
                     'terpenoid_b_pinene_mg_g',
                     'terpenoid_b_pinene_percent',
                     'terpenoid_status',
                     'terpenoid_terpinolene_mg_g',
                     'terpenoid_terpinolene_percent']
                     
                     
lab_results_df.drop(labels = decpreciated_cols, axis = 1, inplace = True)

In [12]:
lab_results_df

,global_id,created_at,mme_id,user_id,external_id,deleted_at,inventory_id,updated_at,status,testing_status,batch_id,parent_lab_result_id,og_parent_lab_result_id,copied_from_lab_id,lab_user_id,tested_at,type,foreign_matter,moisture_content_percent,growth_regulators_ppm,cannabinoid_status,cannabinoid_editor,cannabinoid_d9_thca_percent,cannabinoid_d9_thca_mg_g,cannabinoid_d9_thc_percent,cannabinoid_d9_thc_mg_g,cannabinoid_d8_thc_percent,cannabinoid_d8_thc_mg_g,cannabinoid_cbd_percent,cannabinoid_cbd_mg_g,cannabinoid_cbda_percent,cannabinoid_cbda_mg_g,cannabinoid_cbdv_percent,cannabinoid_cbg_percent,cannabinoid_cbg_mg_g,terpenoid_pinene_percent,terpenoid_pinene_mg_g,microbial_status,microbial_editor,microbial_bile_tolerant_cfu_g,microbial_pathogenic_e_coli_cfu_g,microbial_salmonella_cfu_g,mycotoxin_status,mycotoxin_editor,mycotoxin_aflatoxins_ppb,mycotoxin_ochratoxin_ppb,metal_status,metal_editor,metal_arsenic_ppm,metal_cadmium_ppm,metal_lead_ppm,metal_mercury_ppm,pesticide_status,pesticide_editor,pesticide_abamectin_ppm,pesticide_acequinocyl_ppm,pesticide_bifenazate_ppm,pesticide_cyfluthrin_ppm,pesticide_cypermethrin_ppm,pesticide_etoxazole_ppm,pesticide_flonicamid_ppm,pesticide_fludioxonil_ppm,pesticide_imidacloprid_ppm,pesticide_myclobutanil_ppm,pesticide_spinosad_ppm,pesticide_spirotetramet_ppm,pesticide_thiamethoxam_ppm,pesticide_trifloxystrobin_ppm,solvent_status,solvent_editor,solvent_butanes_ppm,solvent_heptane_ppm,solvent_propane_ppm,notes,thc_percent,intermediate_type,moisture_content_water_activity_rate,solvent_acetone_ppm,solvent_benzene_ppm,solvent_cyclohexane_ppm,solvent_chloroform_ppm,solvent_dichloromethane_ppm,solvent_ethyl_acetate_ppm,solvent_hexanes_ppm,solvent_isopropanol_ppm,solvent_methanol_ppm,solvent_pentanes_ppm,solvent_toluene_ppm,solvent_xylene_ppm,pesticide_acephate_ppm,pesticide_acetamiprid_ppm,pesticide_aldicarb_ppm,pesticide_azoxystrobin_ppm,pesticide_bifenthrin_ppm,pesticide_boscalid_ppm,pesticide_carbaryl_ppm,pesticide_carbofuran_ppm,pesticide_chlorantraniliprole_ppm,pesticide_chlorfenapyr_ppm,pesticide_chlorpyrifos_ppm,pesticide_clofentezine_ppm,pesticide_daminozide_ppm,pesticide_ddvp_dichlorvos_ppm,pesticide_diazinon_ppm,pesticide_dimethoate_ppm,pesticide_ethoprophos_ppm,pesticide_etofenprox_ppm,pesticide_fenoxycarb_ppm,pesticide_fenpyroximate_ppm,pesticide_fipronil_ppm,pesticide_hexythiazox_ppm,pesticide_imazalil_ppm,pesticide_kresoxim_methyl_ppm,pesticide_malathion_ppm,pesticide_metalaxyl_ppm,pesticide_methiocarb_ppm,pesticide_methomyl_ppm,pesticide_methyl_parathion_ppm,pesticide_mgk_264_ppm,pesticide_naled_ppm,pesticide_oxamyl_ppm,pesticide_paclobutrazol_ppm,pesticide_permethrinsa_ppm,pesticide_phosmet_ppm,pesticide_prallethrin_ppm,pesticide_propiconazole_ppm,pesticide_propoxur_ppm,pesticide_pyrethrinsbc_ppm,pesticide_pyridaben_ppm,pesticide_spiromesifen_ppm,pesticide_spirotetramat_ppm,pesticide_spiroxamine_ppm,pesticide_tebuconazole_ppm,pesticide_thiacloprid_ppm,foreign_matter_stems,foreign_matter_seeds,test_for_terpenes
0,NaN,2018-01-31 22:34:19,WAWA1.MMCY,WAWA1.US29D,5275985106676286.0,NaT,WAJ412964.INCP,2018-01-31 22:34:19,not tested,completed,WAJ412964.BA1JS,NaN,NaN,NaN,WAWA1.US29D,2016-04-22 02:30:00,intermediate_product,False,0.0,0.0,completed,NaN,75.60,NaN,6.80,NaN,0.0,NaN,2.90,NaN,0.00,NaN,NaN,0.0,NaN,NaN,NaN,completed,WAWA1.US1,0.0,0.0,NaN,completed,WAWA1.US1,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,completed,WAWA1.US1,0.0,0.0,0.0,NaN,76.000,hydrocarbon_concentrate,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False
1,WAJ412964.LR2,2018-01-31 22:36:20,WAWA1.MMCY,WAWA1.US29D,4129640000001061.0,NaT,WAJ412964.ING1,2018-01-31 22:36:20,passed,completed,WAJ412964.BA1LC,NaN,NaN,NaN,WAWA1.US29D,2017-11-13 00:36:00,harvest_materials,False,4.9,0.